In [1]:
import shelve

## Code Shelf

In [2]:
shelf = shelve.open('CodeShelf')

## Add or Update

In [3]:
value = """
! apt-get install -y xvfb python-opengl
! pip install pyvirtualdisplay

from pyvirtualdisplay import Display

display = Display(visible=0, size=(200, 150))
display.start()
"""

key = 'openai_virtual_display'

print(shelf.get(key))
shelf[key] = value
shelf.sync()


! apt-get install -y xvfb python-opengl
! pip install pyvirtualdisplay

from pyvirtualdisplay import Display

display = Display(visible=0, size=(200, 150))
display.start()



## Enumerate

In [4]:
keys = list(shelf.keys())
print(len(keys), keys)

14 ['plot_history', 'model_summary', 'run_summary', 'tf_imports', 'basic_image_augmentations', 'image_randaugment', 'run_summary_multioutput', 'show_dataset_samples', 'draw_bb_boxes', 'dense_boxes', 'save_frames_as_gif', 'display_gifs', 'display_gifs_ipython', 'openai_virtual_display']


## Query

In [5]:
print(shelf['display_gifs'])


from IPython.display import HTML, display
from PIL import Image

def display_episodes(sims, episodes):
    html_preable = '<div style="display: flex; flex-wrap: wrap;">'
    html_postamble = '</div>'
    html_body = ''
    for episode_id in episodes:
        content_preamble = '<div style="margin: 0 .2%;  width:20%">'
        content_postamble = '</div>'
        gif_name = sims[episode_id]

        content = '<img src="{}"/>
<p style="text-align: center">Episode: {}</p>'.format(gif_name, episode_id)
        html_body += content_preamble + content + content_postamble

    display(HTML(html_preable + html_body + html_postamble))

display_episodes(sims, range(episodes)[:15])

